In [13]:
#@title (Run) Part 1: Define required functions for Data processing

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torch.backends.cudnn as cudnn
from torch.autograd import Variable
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from torch import Tensor
import os
import argparse
class MyDataset(torch.utils.data.Dataset):
    def __init__(self, data, targets):
        self.data = data
        self.targets = targets

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        return self.data[index], self.targets[index]

def balance_val_split(dataset, train_size=12500):

    try:
        targets = np.array(dataset.targets)
    except:
        targets = []  # create an empty list to store the targets
        for data in dataset.datasets:
            targets += data.targets  # concatenate the targets from each dataset into the list
        targets = np.array(targets)
    #targets = np.array(dataset.datasets.targets)
    train_indices, val_indices = train_test_split(
        np.arange(targets.shape[0]),
        train_size=train_size,
        stratify=targets
    )
    train_dataset = Subset(dataset, indices=train_indices)
    # Get the data from the subset dataset
    subset_data = [train_dataset[idx][0] for idx in range(len(train_dataset))]
    subset_labels = [train_dataset[idx][1] for idx in range(len(train_dataset))]
    # Create a dataset from the list of data and targets
    train_dataset = MyDataset(subset_data, subset_labels)
    
    
    val_dataset = Subset(dataset, indices=val_indices)
    # Get the data from the subset dataset
    subset_data = [val_dataset[idx][0] for idx in range(len(val_dataset))]
    subset_labels = [val_dataset[idx][1] for idx in range(len(val_dataset))]
    # Create a dataset from the list of data and targets
    val_dataset = MyDataset(subset_data, subset_labels)

    return train_dataset, val_dataset


def count_label_frequency(target_train_dataset):
	from collections import Counter
	target_labels = []  # create an empty list to store the labels

	for i in range(len(target_train_dataset)):
			_, label = target_train_dataset[i]  # extract the label for the i-th example in the subset
			target_labels.append(label)  # append the label to the 'subset_labels' list


	return Counter(target_labels)
 


def custom_transform(image: Tensor) -> Tensor:
    import random
    # randomly flip horizontally or vertically with 25% chance
    if random.random() < 0.25:
        image = RandomHorizontalFlip(p=1)(image)
    elif random.random() < 0.5:
        image = RandomVerticalFlip(p=1)(image)
    
    # randomly shift the image by 2 pixels to the left or right with 25% chance
    if random.random() < 0.25:
        image = RandomCrop((image.shape[-2], image.shape[-1] - 2), pad_if_needed=False)(image)
    elif random.random() < 0.5:
        image = RandomCrop((image.shape[-2], image.shape[-1] + 2), pad_if_needed=False)(image)
        
    # randomly shift the image by 2 pixels to the top or bottom with 25% chance
    if random.random() < 0.25:
        image = RandomCrop((image.shape[-2] - 2, image.shape[-1]), pad_if_needed=False)(image)
    elif random.random() < 0.5:
        image = RandomCrop((image.shape[-2] + 2, image.shape[-1]), pad_if_needed=False)(image)

    return image

In [8]:
#@title (Run) Part 2: Define required functions for Data Training

# Training
def train(trainloader, epoch, batch_size=128, logfile = "train.summary"):
    print('\nEpoch: %d' % epoch)
    net.train()
    train_loss = 0
    correct = 0
    total = 0

    for batch_idx, (inputs, targets) in enumerate(trainloader):
        #inputs, targets = inputs.to(device), targets.to(device)
        inputs, targets = inputs.cuda(), targets.cuda()
        optimizer.zero_grad()
        if inputs.shape[0] != batch_size:
          print(inputs.shape)
          continue
        outputs = net(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        #torch.nn.utils.clip_grad_norm_(net.parameters(), 10)
        optimizer.step()

        train_loss += loss.item()
        _, predicted = outputs.max(1)
        total += targets.size(0)
        correct += predicted.eq(targets).sum().item()

        if batch_idx % 30 == 0:
                print(batch_idx, len(trainloader), 'Train Loss: %.3f | Train Acc: %.3f%% (%d/%d)'
                     % (train_loss/(batch_idx+1), 100.*correct/total, correct, total))
    print(len(trainloader), 'Epoch: %d | Train Loss: %.3f | Train Acc: %.3f%% (%d/%d)'
                     % (epoch, train_loss/(batch_idx+1), 100.*correct/total, correct, total))
    f = open(logfile, "a")
    f.write('Epoch: %d | Train Loss: %.3f | Train Acc: %.3f%% (%d/%d)\n'
                     % (epoch, train_loss/(batch_idx+1), 100.*correct/total, correct, total))
    f.close()

def test(testloader, epoch, batch_size=128, logfile = "train.summary", save_modelpath = './DLA'):
    global best_acc
    net.eval()
    test_loss = 0
    correct = 0
    total = 0



    with torch.no_grad():
        for batch_idx, (inputs, targets) in enumerate(testloader):
            #inputs, targets = inputs.to(device), targets.to(device)
            inputs, targets = inputs.cuda(), targets.cuda()
            outputs = net(inputs)
            loss = criterion(outputs, targets)

            test_loss += loss.item()
            _, predicted = outputs.max(1)
            total += targets.size(0)
            correct += predicted.eq(targets).sum().item()

            if batch_idx % 30 == 0:
                print(batch_idx, len(testloader), 'Test Loss: %.3f | Test Acc: %.3f%% (%d/%d)'
                         % (test_loss/(batch_idx+1), 100.*correct/total, correct, total))
    print(len(testloader), 'Epoch: %d | Test Loss: %.3f | Test Acc: %.3f%% (%d/%d)'
                         % (epoch, test_loss/(batch_idx+1), 100.*correct/total, correct, total))
    f = open(logfile, "a")
    f.write('Epoch: %d | Test Loss: %.3f | Test Acc: %.3f%% (%d/%d)\n'
                         % (epoch, test_loss/(batch_idx+1), 100.*correct/total, correct, total))
    f.close()
    # Save checkpoint.
    acc = 100.*correct/total
    if acc > best_acc:
        print('Saving..')
        state = {
            'net': net.state_dict(),
            'acc': acc,
            'epoch': epoch,
        }
        if not os.path.isdir(save_modelpath):
            os.mkdir(save_modelpath)
        torch.save(state, save_modelpath+'/ckpt.pth')
        best_acc = acc

def draw_training_summary(filepath = 'target_train_DCA-BiLSTM.summary'):
    import matplotlib.pyplot as plt
    import numpy as np

    with open(filepath, 'r') as f:
        results_summary = f.read()

    train_epoch = []
    train_loss = []
    train_acc = []
    test_epoch = []
    test_loss=[]
    test_acc=[]
    for line in results_summary.split("\n"):
        try:
            r_epoch = line.split('|')[0].strip().split(' ')[1]
            r_loss = line.split('|')[1].strip().split(' ')[2].replace('%','')
            r_acc = line.split('|')[2].strip().split(' ')[2].replace('%','')
            if 'Train' in line:
                train_epoch.append(int(r_epoch))
                train_loss.append(float(r_loss))
                train_acc.append(float(r_acc))
            if 'Test' in line:
                test_epoch.append(int(r_epoch))
                test_loss.append(float(r_loss))
                test_acc.append(float(r_acc))
        except:
            print(line)

    # Create a new figure and plot the data
    plt.figure(figsize=(12,4))

    plt.subplot(1,2,1)
    plt.plot(train_acc, label='Train')
    plt.plot(test_acc, label='Test')
    plt.axhline(y=np.max(test_acc), color='r', linestyle='--')
    # Add text for the horizontal line
    plt.text(test_epoch[-10], np.max(test_acc)*1.05, np.max(test_acc), color='r', fontsize=10)
    # Customize the plot
    plt.title('Accuracy Curve')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.legend()

    plt.subplot(1,2,2)
    plt.plot(train_loss, label='Train')
    plt.plot(test_loss, label='Test')

    # Customize the plot
    plt.title('Loss Curve')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()



    # Display the plot
    plt.show()

In [9]:
#@title (Run) Part 3: Prepare Cifar10 dataset for target and shadow model
classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')


def create_cifar_dataset_torch(batch_size=128, target_train_size = 15000, target_test_size= 15000, shadow_train_size = 15000, shadow_test_size= 15000):

  # Data
  print('==> Preparing data..')


  transform = transforms.Compose([
      transforms.ToTensor()
  ])
  
  cifar_trainset = torchvision.datasets.CIFAR10(
      root='./data', train=True, download=True, transform=transform)


  cifar_testset = torchvision.datasets.CIFAR10(
      root='./data', train=False, download=True, transform=transform)

  cifar_dataset = torch.utils.data.ConcatDataset([cifar_trainset, cifar_testset])


  #target_train_size = int(0.25 * len(cifar_dataset)) # 15000
  remain_size = len(cifar_dataset) - target_train_size
  target_train_dataset, remain_dataset = torch.utils.data.random_split(cifar_dataset, [target_train_size, remain_size])

  #target_test_size = int(0.25 * len(cifar_dataset)) # 15000
  remain_size = len(remain_dataset) - target_test_size
  target_test_dataset, remain_dataset = torch.utils.data.random_split(remain_dataset, [target_test_size, remain_size])

  #target_test_dataset, remain_dataset = balance_val_split(remain_dataset, train_size=target_test_size)


  #shadow_train_size = int(0.25 * len(cifar_dataset)) # 15000
  remain_size = len(remain_dataset) - shadow_train_size
  shadow_train_dataset, shadow_test_dataset = torch.utils.data.random_split(remain_dataset, [shadow_train_size, remain_size])
  #shadow_train_dataset, shadow_test_dataset = balance_val_split(remain_dataset, train_size=shadow_train_size)

  print("Setting target_train_dataset size to ",len(target_train_dataset), count_label_frequency(target_train_dataset))
  print("Setting target_test_dataset size to ",len(target_test_dataset), count_label_frequency(target_test_dataset))
  print("Setting shadow_train_dataset size to ",len(shadow_train_dataset), count_label_frequency(shadow_train_dataset))
  print("Setting shadow_test_dataset size to ",len(shadow_test_dataset), count_label_frequency(shadow_test_dataset))
  #print("Setting testset size to ",len(testset))



  '''
  transform_train = transforms.Compose([
      transforms.RandomCrop(32, padding=4),
      transforms.RandomHorizontalFlip(),
      transforms.ToTensor(),
      transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
  ])
  '''



  transform_train = transforms.Compose([
      transforms.RandomCrop(32, padding=4),
      custom_transform,
      transforms.ToTensor(),
      transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
  ])

  transform_test = transforms.Compose([
      transforms.ToTensor(),
      transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
  ])

  # apply the data augmentation transformations to the subset
  target_train_dataset.dataset.transform = transform_train
  # Load the transformed subset using a DataLoader
  target_trainloader = DataLoader(target_train_dataset, batch_size=batch_size, shuffle=True, drop_last = True)


  target_test_dataset.dataset.transform = transform_test
  # Load the transformed subset using a DataLoader
  target_testloader = DataLoader(target_test_dataset, batch_size=batch_size, shuffle=True, drop_last = True)


  # apply the data augmentation transformations to the subset
  shadow_train_dataset.dataset.transform = transform_train
  # Load the transformed subset using a DataLoader
  shadow_trainloader = DataLoader(shadow_train_dataset, batch_size=batch_size, shuffle=True, drop_last = True)


  shadow_test_dataset.dataset.transform = transform_test
  # Load the transformed subset using a DataLoader
  shadow_testloader = DataLoader(shadow_test_dataset, batch_size=batch_size, shuffle=True, drop_last = True)

  return target_trainloader, target_testloader, shadow_trainloader, shadow_testloader


In [10]:
#@title (Run) Part 4.2: Define required functions for DLA & DLA+RNN

import torch
import torch.nn as nn
import torch.nn.functional as F


class BasicBlock(nn.Module):
    expansion = 1

    def __init__(self, in_planes, planes, stride=1):
        super(BasicBlock, self).__init__()
        self.conv1 = nn.Conv2d(
            in_planes, planes, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(planes)
        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3,
                               stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(planes)

        self.shortcut = nn.Sequential()
        if stride != 1 or in_planes != self.expansion*planes:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_planes, self.expansion*planes,
                          kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(self.expansion*planes)
            )

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))
        out += self.shortcut(x)
        out = F.relu(out)
        return out

class Root(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size=1):
        super(Root, self).__init__()
        self.conv = nn.Conv2d(
            in_channels, out_channels, kernel_size,
            stride=1, padding=(kernel_size - 1) // 2, bias=False)
        self.bn = nn.BatchNorm2d(out_channels)

    def forward(self, xs):
        x = torch.cat(xs, 1)
        out = F.relu(self.bn(self.conv(x)))
        return out


class Tree(nn.Module):
    def __init__(self, block, in_channels, out_channels, level=1, stride=1):
        super(Tree, self).__init__()
        self.level = level
        if level == 1:
            self.root = Root(2*out_channels, out_channels)
            self.left_node = block(in_channels, out_channels, stride=stride)
            self.right_node = block(out_channels, out_channels, stride=1)
        else:
            self.root = Root((level+2)*out_channels, out_channels)
            for i in reversed(range(1, level)):
                subtree = Tree(block, in_channels, out_channels,
                               level=i, stride=stride)
                self.__setattr__('level_%d' % i, subtree)
            self.prev_root = block(in_channels, out_channels, stride=stride)
            self.left_node = block(out_channels, out_channels, stride=1)
            self.right_node = block(out_channels, out_channels, stride=1)

    def forward(self, x):
        xs = [self.prev_root(x)] if self.level > 1 else []
        for i in reversed(range(1, self.level)):
            level_i = self.__getattr__('level_%d' % i)
            x = level_i(x)
            xs.append(x)
        x = self.left_node(x)
        xs.append(x)
        x = self.right_node(x)
        xs.append(x)
        out = self.root(xs)
        return out


class DLA(nn.Module):
    def __init__(self, block=BasicBlock, num_classes=10, enable_RNN='LSTM'):
        super(DLA, self).__init__()
        
        self.enable_RNN = enable_RNN
        if enable_RNN not in ['None', 'LSTM', 'Bi-LSTM']:
          raise Exception("enable_RNN only supports one of ['None', 'LSTM', 'Bi-LSTM']")
        
        self.base = nn.Sequential(
            nn.Conv2d(3, 16, kernel_size=3, stride=1, padding=1, bias=False),
            nn.BatchNorm2d(16),
            nn.ReLU(True)
        )

        self.layer1 = nn.Sequential(
            nn.Conv2d(16, 16, kernel_size=3, stride=1, padding=1, bias=False),
            nn.BatchNorm2d(16),
            nn.ReLU(True)
        )

        self.layer2 = nn.Sequential(
            nn.Conv2d(16, 32, kernel_size=3, stride=1, padding=1, bias=False),
            nn.BatchNorm2d(32),
            nn.ReLU(True)
        )

        self.layer3 = Tree(block,  32,  64, level=1, stride=1)
        self.layer4 = Tree(block,  64, 128, level=2, stride=2)
        self.layer5 = Tree(block, 128, 256, level=2, stride=2)
        self.layer6 = Tree(block, 256, 512, level=1, stride=2)
        self.linear = nn.Linear(4096, num_classes)
        
        if enable_RNN == 'None':
          self.linear = nn.Linear(512, num_classes)
        elif enable_RNN == 'LSTM':
          self.rnn = nn.LSTM(512, 1024, dropout = 0.6)
          self.linear = nn.Linear(1024, num_classes)
        elif enable_RNN == 'Bi-LSTM':
          self.rnn = nn.LSTM(512, 2048, 1, dropout = 0.3, batch_first=True, bidirectional=True)
          self.linear = nn.Linear(4096, num_classes)
        else:
          raise Exception("enable_RNN only supports one of ['None', 'LSTM', 'Bi-LSTM']")

    def forward(self, x):
        out = self.base(x)
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)
        out = self.layer5(out)
        out = self.layer6(out)
        out = F.avg_pool2d(out, 4)

        if self.enable_RNN == 'None':
          out = out.view(out.size(0), -1)
          out = self.linear(out)
        else:
          # add LSTM
          #print(out.shape)
          out = out.view(out.size(0), 1,  -1)
          out,_ = self.rnn(out)
          out = out.view(out.size(0), -1)
          out = self.linear(out)
        return out

In [11]:
#@title (Run) Part 5.1.1: Select an architecture for Target model training
#from torchsummary import summary


#@markdown Enter the folder of path to the pdb files:
#@markdown * Option 1: ReNet
#@markdown * Option 2: DLA
#@markdown * Option 3: DLA-LSTM
#@markdown * Option 4: DLA-BiLSTM


method_name = 'DLA-BiLSTM'  #@param {type:"string"}
save_model_folder = './Target-DLA-BiLSTM_models/'   #@param {type:"string"}
batch_size = 64  #@param {type:"integer"}
load_pretrain_weight = False   #@param {type:"boolean"}


print('==> Building model for ' + method_name)
if method_name == 'DLA-BiLSTM':
  # Model
  
  net = DLA(num_classes=10, enable_RNN='Bi-LSTM')
  net.cuda()
elif method_name == 'DLA-LSTM':
  # Model
  
  net = DLA(num_classes=10, enable_RNN='LSTM')
  net.cuda()
elif method_name == 'DLA':
  # Model
  
  net = DLA(num_classes=10, enable_RNN='None')
  net.cuda()


device = 'cuda' if torch.cuda.is_available() else 'cpu'
  
if load_pretrain_weight:
  try:
      # Load checkpoint.
      print('==> Resuming from checkpoint..')
      checkpoint = torch.load(save_model_folder+'/ckpt.pth')
      net.load_state_dict(checkpoint['net'])
      best_acc = checkpoint['acc']
      start_epoch = checkpoint['epoch']
  except:
      print('!!! Error: no checkpoint directory found!')
else:
  print('==> Training model from scratch..')
  
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.01,
                      momentum=0.9, weight_decay=5e-4)
#optimizer = torch.optim.Adam(net.parameters(), lr=0.01)
#scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=200)

pytorch_total_params = sum(p.numel() for p in net.parameters() if p.requires_grad)
print("Total trained parameters: ",pytorch_total_params)

==> Building model for DLA-BiLSTM
==> Training model from scratch..
Total trained parameters:  58303034


In [14]:
#@title (Run) Part 5.1.2: Setup Target and Shadow datasets for ReNet Training

target_train_size = 15000 #@param {type:"integer"}
target_test_size= 15000 #@param {type:"integer"}
shadow_train_size = 15000  #@param {type:"integer"} 
shadow_test_size= 15000 #@param {type:"integer"}

# create dataset for renet 
print('==> Preparing dataset..')
target_trainloader, target_testloader, shadow_trainloader, shadow_testloader = create_cifar_dataset_torch(batch_size=batch_size, target_train_size = target_train_size, target_test_size= target_test_size, shadow_train_size = shadow_train_size, shadow_test_size= shadow_test_size)

==> Preparing dataset..
==> Preparing data..
Files already downloaded and verified
Files already downloaded and verified
Setting target_train_dataset size to  15000 Counter({0: 1548, 1: 1545, 9: 1509, 2: 1507, 5: 1503, 7: 1501, 3: 1499, 4: 1485, 6: 1479, 8: 1424})
Setting target_test_dataset size to  15000 Counter({8: 1558, 2: 1550, 5: 1527, 7: 1502, 6: 1499, 9: 1499, 0: 1496, 4: 1493, 3: 1438, 1: 1438})
Setting shadow_train_dataset size to  15000 Counter({8: 1555, 7: 1521, 3: 1511, 5: 1505, 2: 1499, 4: 1492, 6: 1489, 9: 1483, 1: 1474, 0: 1471})
Setting shadow_test_dataset size to  15000 Counter({3: 1552, 1: 1543, 6: 1533, 4: 1530, 9: 1509, 0: 1485, 7: 1476, 5: 1465, 8: 1463, 2: 1444})


In [ ]:
#@title (Run) Part 5.2.3: Start Target model training

max_epoch = 50  #@param {type:"integer"}
train_result_summary = 'target_train_DCA-BiLSTM.summary'   #@param {type:"string"}

best_acc = 0  # best test accuracy
start_epoch = 0  # start from epoch 0 or last checkpoint epoch


if not load_pretrain_weight:
    f = open(train_result_summary, "w")
    f.write('')
    f.close()

for epoch in range(start_epoch, start_epoch+max_epoch):
    train(target_trainloader, epoch, batch_size=batch_size, logfile = train_result_summary)
    test(target_testloader, epoch, batch_size=batch_size, logfile = train_result_summary, save_modelpath = save_model_folder)
    #scheduler.step()


Epoch: 0
0 234 Train Loss: 2.302 | Train Acc: 6.250% (4/64)
30 234 Train Loss: 2.283 | Train Acc: 12.097% (240/1984)
60 234 Train Loss: 2.235 | Train Acc: 14.139% (552/3904)
90 234 Train Loss: 2.170 | Train Acc: 15.831% (922/5824)
120 234 Train Loss: 2.113 | Train Acc: 17.704% (1371/7744)
150 234 Train Loss: 2.063 | Train Acc: 19.474% (1882/9664)
180 234 Train Loss: 2.031 | Train Acc: 20.666% (2394/11584)
210 234 Train Loss: 1.996 | Train Acc: 22.082% (2982/13504)
234 Epoch: 0 | Train Loss: 1.972 | Train Acc: 22.796% (3414/14976)
0 234 Test Loss: 2.071 | Test Acc: 23.438% (15/64)
30 234 Test Loss: 1.983 | Test Acc: 22.782% (452/1984)
60 234 Test Loss: 2.010 | Test Acc: 21.593% (843/3904)
90 234 Test Loss: 2.012 | Test Acc: 21.703% (1264/5824)
120 234 Test Loss: 2.014 | Test Acc: 21.501% (1665/7744)
150 234 Test Loss: 2.011 | Test Acc: 21.720% (2099/9664)
180 234 Test Loss: 2.007 | Test Acc: 21.892% (2536/11584)
210 234 Test Loss: 2.006 | Test Acc: 21.875% (2954/13504)
234 Epoch: 0 | T

120 234 Test Loss: 1.336 | Test Acc: 62.203% (4817/7744)
150 234 Test Loss: 1.331 | Test Acc: 62.459% (6036/9664)
180 234 Test Loss: 1.320 | Test Acc: 62.586% (7250/11584)
210 234 Test Loss: 1.308 | Test Acc: 62.930% (8498/13504)
234 Epoch: 7 | Test Loss: 1.310 | Test Acc: 62.894% (9419/14976)

Epoch: 8
0 234 Train Loss: 0.342 | Train Acc: 87.500% (56/64)
30 234 Train Loss: 0.360 | Train Acc: 88.004% (1746/1984)
60 234 Train Loss: 0.322 | Train Acc: 89.575% (3497/3904)
90 234 Train Loss: 0.323 | Train Acc: 89.475% (5211/5824)
120 234 Train Loss: 0.328 | Train Acc: 89.140% (6903/7744)
150 234 Train Loss: 0.332 | Train Acc: 88.959% (8597/9664)
180 234 Train Loss: 0.330 | Train Acc: 88.968% (10306/11584)
210 234 Train Loss: 0.338 | Train Acc: 88.626% (11968/13504)
234 Epoch: 8 | Train Loss: 0.346 | Train Acc: 88.201% (13209/14976)
0 234 Test Loss: 1.086 | Test Acc: 60.938% (39/64)
30 234 Test Loss: 1.371 | Test Acc: 61.996% (1230/1984)
60 234 Test Loss: 1.375 | Test Acc: 61.424% (2398/390

234 Epoch: 15 | Train Loss: 0.080 | Train Acc: 97.396% (14586/14976)
0 234 Test Loss: 1.203 | Test Acc: 70.312% (45/64)
30 234 Test Loss: 1.158 | Test Acc: 71.119% (1411/1984)
60 234 Test Loss: 1.178 | Test Acc: 70.441% (2750/3904)
90 234 Test Loss: 1.154 | Test Acc: 71.240% (4149/5824)
120 234 Test Loss: 1.167 | Test Acc: 71.100% (5506/7744)
150 234 Test Loss: 1.171 | Test Acc: 71.254% (6886/9664)
180 234 Test Loss: 1.167 | Test Acc: 71.288% (8258/11584)
210 234 Test Loss: 1.173 | Test Acc: 71.097% (9601/13504)
234 Epoch: 15 | Test Loss: 1.167 | Test Acc: 71.354% (10686/14976)

Epoch: 16
0 234 Train Loss: 0.040 | Train Acc: 98.438% (63/64)
30 234 Train Loss: 0.046 | Train Acc: 98.538% (1955/1984)
60 234 Train Loss: 0.052 | Train Acc: 98.412% (3842/3904)
90 234 Train Loss: 0.057 | Train Acc: 98.231% (5721/5824)
120 234 Train Loss: 0.060 | Train Acc: 98.166% (7602/7744)
150 234 Train Loss: 0.063 | Train Acc: 98.034% (9474/9664)
180 234 Train Loss: 0.064 | Train Acc: 97.980% (11350/11584

90 234 Train Loss: 0.029 | Train Acc: 99.159% (5775/5824)
120 234 Train Loss: 0.031 | Train Acc: 99.019% (7668/7744)
150 234 Train Loss: 0.032 | Train Acc: 98.934% (9561/9664)
180 234 Train Loss: 0.032 | Train Acc: 98.930% (11460/11584)
210 234 Train Loss: 0.031 | Train Acc: 98.963% (13364/13504)
234 Epoch: 23 | Train Loss: 0.030 | Train Acc: 99.012% (14828/14976)
0 234 Test Loss: 1.153 | Test Acc: 71.875% (46/64)
30 234 Test Loss: 1.036 | Test Acc: 74.143% (1471/1984)
60 234 Test Loss: 1.028 | Test Acc: 74.693% (2916/3904)
90 234 Test Loss: 1.029 | Test Acc: 74.931% (4364/5824)
120 234 Test Loss: 1.011 | Test Acc: 75.504% (5847/7744)
150 234 Test Loss: 1.009 | Test Acc: 75.404% (7287/9664)
180 234 Test Loss: 1.014 | Test Acc: 75.233% (8715/11584)
210 234 Test Loss: 1.020 | Test Acc: 75.081% (10139/13504)
234 Epoch: 23 | Test Loss: 1.026 | Test Acc: 74.907% (11218/14976)
Saving..

Epoch: 24
0 234 Train Loss: 0.002 | Train Acc: 100.000% (64/64)
30 234 Train Loss: 0.017 | Train Acc: 99.6

In [ ]:
draw_training_summary(filepath = 'target_train_DCA-BiLSTM.summary')

In [ ]:
#@title (Run) Part 5.1.1: Select an architecture for Target model training
#from torchsummary import summary


#@markdown Enter the folder of path to the pdb files:
#@markdown * Option 1: ReNet
#@markdown * Option 2: DLA
#@markdown * Option 3: DLA-LSTM
#@markdown * Option 4: DLA-BiLSTM


method_name = 'DLA-BiLSTM'  #@param {type:"string"}
save_model_folder = './Shadow-DLA-BiLSTM_models/'   #@param {type:"string"}
batch_size = 64  #@param {type:"integer"}
load_pretrain_weight = False   #@param {type:"boolean"}


print('==> Building model for ' + method_name)
if method_name == 'DLA-BiLSTM':
  # Model
  
  net = DLA(num_classes=10, enable_RNN='Bi-LSTM')
  net.cuda()
elif method_name == 'DLA-LSTM':
  # Model
  
  net = DLA(num_classes=10, enable_RNN='LSTM')
  net.cuda()
elif method_name == 'DLA':
  # Model
  
  net = DLA(num_classes=10, enable_RNN='None')
  net.cuda()


device = 'cuda' if torch.cuda.is_available() else 'cpu'
  
if load_pretrain_weight:
  try:
      # Load checkpoint.
      print('==> Resuming from checkpoint..')
      checkpoint = torch.load(save_model_folder+'/ckpt.pth')
      net.load_state_dict(checkpoint['net'])
      best_acc = checkpoint['acc']
      start_epoch = checkpoint['epoch']
  except:
      print('!!! Error: no checkpoint directory found!')
else:
  print('==> Training model from scratch..')
  
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.01,
                      momentum=0.9, weight_decay=5e-4)
#optimizer = torch.optim.Adam(net.parameters(), lr=0.01)
#scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=200)

pytorch_total_params = sum(p.numel() for p in net.parameters() if p.requires_grad)
print("Total trained parameters: ",pytorch_total_params)

In [ ]:
#@title (Run) Part 5.2.3: Start Shadow model training

max_epoch = 50  #@param {type:"integer"}
train_result_summary = 'shadow_train_DCA-BiLSTM.summary'   #@param {type:"string"}

best_acc = 0  # best test accuracy
start_epoch = 0  # start from epoch 0 or last checkpoint epoch


if not load_pretrain_weight:
    f = open(train_result_summary, "w")
    f.write('')
    f.close()

for epoch in range(start_epoch, start_epoch+max_epoch):
    train(shadow_trainloader, epoch, batch_size=batch_size, logfile = train_result_summary)
    test(shadow_testloader, epoch, batch_size=batch_size, logfile = train_result_summary, save_modelpath = save_model_folder)
    #scheduler.step()

In [ ]:
draw_training_summary(filepath = 'shadow_train_DCA-BiLSTM.summary')

In [ ]:
#@title (Run) Part 5.1.1: Select an architecture for Target model training
#from torchsummary import summary


#@markdown Enter the folder of path to the pdb files:
#@markdown * Option 1: ReNet
#@markdown * Option 2: DLA
#@markdown * Option 3: DLA-LSTM
#@markdown * Option 4: DLA-BiLSTM


method_name = 'DLA-LSTM'  #@param {type:"string"}
save_model_folder = './Target-DLA-LSTM_models/'   #@param {type:"string"}
batch_size = 64  #@param {type:"integer"}
load_pretrain_weight = False   #@param {type:"boolean"}


print('==> Building model for ' + method_name)
if method_name == 'DLA-BiLSTM':
  # Model
  
  net = DLA(num_classes=10, enable_RNN='Bi-LSTM')
  net.cuda()
elif method_name == 'DLA-LSTM':
  # Model
  
  net = DLA(num_classes=10, enable_RNN='LSTM')
  net.cuda()
elif method_name == 'DLA':
  # Model
  
  net = DLA(num_classes=10, enable_RNN='None')
  net.cuda()


device = 'cuda' if torch.cuda.is_available() else 'cpu'
  
if load_pretrain_weight:
  try:
      # Load checkpoint.
      print('==> Resuming from checkpoint..')
      checkpoint = torch.load(save_model_folder+'/ckpt.pth')
      net.load_state_dict(checkpoint['net'])
      best_acc = checkpoint['acc']
      start_epoch = checkpoint['epoch']
  except:
      print('!!! Error: no checkpoint directory found!')
else:
  print('==> Training model from scratch..')
  
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.01,
                      momentum=0.9, weight_decay=5e-4)
#optimizer = torch.optim.Adam(net.parameters(), lr=0.01)
#scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=200)

pytorch_total_params = sum(p.numel() for p in net.parameters() if p.requires_grad)
print("Total trained parameters: ",pytorch_total_params)

In [ ]:
#@title (Run) Part 5.2.3: Start Target model training

max_epoch = 30  #@param {type:"integer"}
train_result_summary = 'target_train_DCA-LSTM.summary'   #@param {type:"string"}

best_acc = 0  # best test accuracy
start_epoch = 0  # start from epoch 0 or last checkpoint epoch


if not load_pretrain_weight:
    f = open(train_result_summary, "w")
    f.write('')
    f.close()

for epoch in range(start_epoch, start_epoch+max_epoch):
    train(target_trainloader, epoch, batch_size=batch_size, logfile = train_result_summary)
    test(target_testloader, epoch, batch_size=batch_size, logfile = train_result_summary, save_modelpath = save_model_folder)
    #scheduler.step()

In [ ]:
draw_training_summary(filepath = 'target_train_DCA-LSTM.summary')

In [ ]:
#@title (Run) Part 5.1.1: Select an architecture for Target model training
#from torchsummary import summary


#@markdown Enter the folder of path to the pdb files:
#@markdown * Option 1: ReNet
#@markdown * Option 2: DLA
#@markdown * Option 3: DLA-LSTM
#@markdown * Option 4: DLA-BiLSTM


method_name = 'DLA-LSTM'  #@param {type:"string"}
save_model_folder = './Shadow-DLA-LSTM_models/'   #@param {type:"string"}
batch_size = 64  #@param {type:"integer"}
load_pretrain_weight = False   #@param {type:"boolean"}


print('==> Building model for ' + method_name)
if method_name == 'DLA-BiLSTM':
  # Model
  
  net = DLA(num_classes=10, enable_RNN='Bi-LSTM')
  net.cuda()
elif method_name == 'DLA-LSTM':
  # Model
  
  net = DLA(num_classes=10, enable_RNN='LSTM')
  net.cuda()
elif method_name == 'DLA':
  # Model
  
  net = DLA(num_classes=10, enable_RNN='None')
  net.cuda()


device = 'cuda' if torch.cuda.is_available() else 'cpu'
  
if load_pretrain_weight:
  try:
      # Load checkpoint.
      print('==> Resuming from checkpoint..')
      checkpoint = torch.load(save_model_folder+'/ckpt.pth')
      net.load_state_dict(checkpoint['net'])
      best_acc = checkpoint['acc']
      start_epoch = checkpoint['epoch']
  except:
      print('!!! Error: no checkpoint directory found!')
else:
  print('==> Training model from scratch..')
  
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.01,
                      momentum=0.9, weight_decay=5e-4)
#optimizer = torch.optim.Adam(net.parameters(), lr=0.01)
#scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=200)

pytorch_total_params = sum(p.numel() for p in net.parameters() if p.requires_grad)
print("Total trained parameters: ",pytorch_total_params)

In [ ]:
#@title (Run) Part 5.2.3: Start Shadow model training

max_epoch = 50  #@param {type:"integer"}
train_result_summary = 'shadow_train_DCA-LSTM.summary'   #@param {type:"string"}

best_acc = 0  # best test accuracy
start_epoch = 0  # start from epoch 0 or last checkpoint epoch


if not load_pretrain_weight:
    f = open(train_result_summary, "w")
    f.write('')
    f.close()

for epoch in range(start_epoch, start_epoch+max_epoch):
    train(shadow_trainloader, epoch, batch_size=batch_size, logfile = train_result_summary)
    test(shadow_testloader, epoch, batch_size=batch_size, logfile = train_result_summary, save_modelpath = save_model_folder)
    #scheduler.step()

In [ ]:
draw_training_summary(filepath = 'shadow_train_DCA-LSTM.summary')

In [ ]:
#@title (Run) Part 5.1.1: Select an architecture for Target model training
#from torchsummary import summary


#@markdown Enter the folder of path to the pdb files:
#@markdown * Option 1: ReNet
#@markdown * Option 2: DLA
#@markdown * Option 3: DLA-LSTM
#@markdown * Option 4: DLA-BiLSTM


method_name = 'DLA'  #@param {type:"string"}
save_model_folder = './Target-DLA_models/'   #@param {type:"string"}
batch_size = 64  #@param {type:"integer"}
load_pretrain_weight = False   #@param {type:"boolean"}


print('==> Building model for ' + method_name)
if method_name == 'DLA-BiLSTM':
  # Model
  
  net = DLA(num_classes=10, enable_RNN='Bi-LSTM')
  net.cuda()
elif method_name == 'DLA-LSTM':
  # Model
  
  net = DLA(num_classes=10, enable_RNN='LSTM')
  net.cuda()
elif method_name == 'DLA':
  # Model
  
  net = DLA(num_classes=10, enable_RNN='None')
  net.cuda()


device = 'cuda' if torch.cuda.is_available() else 'cpu'
  
if load_pretrain_weight:
  try:
      # Load checkpoint.
      print('==> Resuming from checkpoint..')
      checkpoint = torch.load(save_model_folder+'/ckpt.pth')
      net.load_state_dict(checkpoint['net'])
      best_acc = checkpoint['acc']
      start_epoch = checkpoint['epoch']
  except:
      print('!!! Error: no checkpoint directory found!')
else:
  print('==> Training model from scratch..')
  
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.01,
                      momentum=0.9, weight_decay=5e-4)
#optimizer = torch.optim.Adam(net.parameters(), lr=0.01)
#scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=200)

pytorch_total_params = sum(p.numel() for p in net.parameters() if p.requires_grad)
print("Total trained parameters: ",pytorch_total_params)

In [ ]:
#@title (Run) Part 5.2.3: Start Target model training

max_epoch = 50  #@param {type:"integer"}
train_result_summary = 'target_train_DCA.summary'   #@param {type:"string"}

best_acc = 0  # best test accuracy
start_epoch = 0  # start from epoch 0 or last checkpoint epoch


if not load_pretrain_weight:
    f = open(train_result_summary, "w")
    f.write('')
    f.close()

for epoch in range(start_epoch, start_epoch+max_epoch):
    train(target_trainloader, epoch, batch_size=batch_size, logfile = train_result_summary)
    test(target_testloader, epoch, batch_size=batch_size, logfile = train_result_summary, save_modelpath = save_model_folder)
    #scheduler.step()

In [ ]:
draw_training_summary(filepath = 'target_train_DCA.summary')

In [ ]:
#@title (Run) Part 5.1.1: Select an architecture for Target model training
#from torchsummary import summary


#@markdown Enter the folder of path to the pdb files:
#@markdown * Option 1: ReNet
#@markdown * Option 2: DLA
#@markdown * Option 3: DLA-LSTM
#@markdown * Option 4: DLA-BiLSTM


method_name = 'DLA'  #@param {type:"string"}
save_model_folder = './Shadow-DLA_models/'   #@param {type:"string"}
batch_size = 64  #@param {type:"integer"}
load_pretrain_weight = False   #@param {type:"boolean"}


print('==> Building model for ' + method_name)
if method_name == 'DLA-BiLSTM':
  # Model
  
  net = DLA(num_classes=10, enable_RNN='Bi-LSTM')
  net.cuda()
elif method_name == 'DLA-LSTM':
  # Model
  
  net = DLA(num_classes=10, enable_RNN='LSTM')
  net.cuda()
elif method_name == 'DLA':
  # Model
  
  net = DLA(num_classes=10, enable_RNN='None')
  net.cuda()


device = 'cuda' if torch.cuda.is_available() else 'cpu'
  
if load_pretrain_weight:
  try:
      # Load checkpoint.
      print('==> Resuming from checkpoint..')
      checkpoint = torch.load(save_model_folder+'/ckpt.pth')
      net.load_state_dict(checkpoint['net'])
      best_acc = checkpoint['acc']
      start_epoch = checkpoint['epoch']
  except:
      print('!!! Error: no checkpoint directory found!')
else:
  print('==> Training model from scratch..')
  
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.01,
                      momentum=0.9, weight_decay=5e-4)
#optimizer = torch.optim.Adam(net.parameters(), lr=0.01)
#scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=200)

pytorch_total_params = sum(p.numel() for p in net.parameters() if p.requires_grad)
print("Total trained parameters: ",pytorch_total_params)

In [ ]:
#@title (Run) Part 5.2.3: Start Shadow model training

max_epoch = 50  #@param {type:"integer"}
train_result_summary = 'shadow_train_DCA.summary'   #@param {type:"string"}

best_acc = 0  # best test accuracy
start_epoch = 0  # start from epoch 0 or last checkpoint epoch


if not load_pretrain_weight:
    f = open(train_result_summary, "w")
    f.write('')
    f.close()

for epoch in range(start_epoch, start_epoch+max_epoch):
    train(shadow_trainloader, epoch, batch_size=batch_size, logfile = train_result_summary)
    test(shadow_testloader, epoch, batch_size=batch_size, logfile = train_result_summary, save_modelpath = save_model_folder)
    #scheduler.step()

In [ ]:
draw_training_summary(filepath = 'shadow_train_DCA.summary')